In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [16]:
#DepremVerileri.txt dosyasında 1900 yılından günümüze kadar olan 3.5 üzeri büyüklükteki
#Yaklaşık 17857 deprem vardır.
#DepremVerileri.txt dosyasından enlem,boylam, xM (büyüklük) ve Yer bilgilerini alıyoruz
#enlem, boylam, xM, Yer
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
depremRDD = sc.textFile("DepremVerileri.txt")
depremRDD = depremRDD.filter(lambda line : "Deprem Kodu" not in line)
def splitAndTake(line):
  arr = line.split("\t")
  enlem = float(arr[4])
  boylam = float(arr[5])
  buyukluk = float(arr[7])
  yer = arr[14]
  tarih = arr[2]
  return ( tarih,enlem, boylam , buyukluk ,yer)
depremRDD = depremRDD.map(lambda a : splitAndTake(a))
depremRDD = depremRDD.sortBy(lambda b : b[3],ascending=False)
#Türkiye de 1900 yılından beri olmuş en büyük 10 deprem
for d in depremRDD.take(10):
  print(d)

('1939.12.26', 39.8, 39.51, 7.9, 'KURUTILEK- (ERZINCAN) [North East  3.0 km]')
('1926.06.26', 36.54, 27.33, 7.7, 'ONIKI ADALAR (AKDENIZ)')
('1930.05.06', 37.98, 44.48, 7.6, 'T�RKIYE-IRAN SINIR B�LGESI')
('1976.11.24', 39.05, 44.04, 7.5, 'YENIYAKA-CALDIRAN (VAN) [South East  1.9 km]')
('1999.08.17', 40.76, 29.97, 7.4, 'BASISKELE (KOCAELI) [North East  2.0 km]')
('1912.08.09', 40.6, 27.2, 7.3, 'ERIKLICE-SARKOY (TEKIRDAG) [South East  4.3 km]')
('2011.10.23', 38.7212, 43.411, 7.2, 'YEMLICE- (VAN) [North West  1.5 km]')
('1999.11.12', 40.74, 31.21, 7.2, 'UGUR- (DUZCE) [North East  0.3 km]')
('1953.03.18', 39.99, 27.36, 7.2, 'SOGUCAK-YENICE (�ANAKKALE) [South West  2.3 km]')
('1948.02.09', 35.41, 27.2, 7.2, 'AKDENIZ')


In [25]:
#4,5,7,14
#Türkiye yi enlem ve boylamlara bakarak 8 (karesel) bölgeye ayırıp en çok deprem olan bölgeleri bulunuz
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
depremRDD = sc.textFile("DepremVerileri.txt")
depremRDD = depremRDD.filter(lambda line : "Deprem Kodu" not in line)
def splitAndTake(line):
  arr = line.split("\t")
  enlem = float(arr[4])
  boylam = float(arr[5])
  buyukluk = float(arr[7])
  yer = arr[14]
  tarih = arr[2]
  return ( enlem, boylam )
depremRDD = depremRDD.map(lambda a : splitAndTake(a))
def bolgeAta(enlem,boylam):
  bolge = ""
  if (enlem >= 39):
    if (boylam <=31):
      bolge = "Bölge 1"
    elif (boylam <= 36):
      bolge = "Bölge 2"
    elif (boylam <= 41):
      bolge = "Bölge 3"
    else:
      bolge = "Bölge 4"
  else:
     if (boylam <=31):
      bolge = "Bölge 5"
     elif (boylam <= 36):
      bolge = "Bölge 6"
     elif (boylam <= 41):
      bolge = "Bölge 7"
     else:
      bolge = "Bölge 8"
  return (bolge,1)
depremRDD = depremRDD.map(lambda d : bolgeAta(d[0],d[1]))
depremRDD = depremRDD.reduceByKey(lambda x,y : x+y)
depremRDD = depremRDD.sortBy(lambda a : a[1],ascending=False)
depremRDD.take(15)

[('Bölge 5', 8146),
 ('Bölge 1', 2835),
 ('Bölge 8', 1561),
 ('Bölge 7', 1349),
 ('Bölge 6', 1286),
 ('Bölge 2', 1000),
 ('Bölge 3', 996),
 ('Bölge 4', 759)]

In [26]:
s = sc.parallelize([1,2,3,4,5,6,7,8])
toplam = s.reduce(lambda x,y : x+y)
print(toplam)

36


In [27]:
rdd1 = sc.parallelize([("a",1),("b",8),("c",2),("a",4),("c",3)])
rdd2 = sc.parallelize([("a",4),("b",5),("e",1),("f",4),("b",2)])

In [29]:
rdd3 = rdd1.union(rdd2)
print(rdd3.collect())

[('a', 1), ('b', 8), ('c', 2), ('a', 4), ('c', 3), ('a', 4), ('b', 5), ('e', 1), ('f', 4), ('b', 2)]


In [30]:
rdd4 = rdd1.join(rdd2)
print(rdd4.collect())

[('b', (8, 5)), ('b', (8, 2)), ('a', (1, 4)), ('a', (4, 4))]


In [36]:
rdd1 = sc.parallelize([("a",1),("b",8),("c",2),("a",4),("c",3)])
groupRDD = rdd1.groupByKey().map(lambda x : (x[0],tuple(x[1])))
#groupRDD = rdd1.groupByKey()
groupRDD.collect()

[('b', (8,)), ('c', (2, 3)), ('a', (1, 4))]

In [39]:
rdd1 = sc.parallelize([("a",1),("b",8),("c",2),("a",4),("c",3)])
rdd2 = sc.parallelize([("a",4),("b",5),("e",1),("f",4),("b",2)])

rdd5 = rdd1.cartesian(rdd2)
rdd5.collect()
#rdd5.sortBy(lambda x: x[0][0]).collect()

[(('a', 1), ('a', 4)),
 (('a', 1), ('b', 5)),
 (('b', 8), ('a', 4)),
 (('b', 8), ('b', 5)),
 (('a', 1), ('e', 1)),
 (('a', 1), ('f', 4)),
 (('b', 8), ('e', 1)),
 (('b', 8), ('f', 4)),
 (('a', 1), ('b', 2)),
 (('b', 8), ('b', 2)),
 (('c', 2), ('a', 4)),
 (('c', 2), ('b', 5)),
 (('a', 4), ('a', 4)),
 (('a', 4), ('b', 5)),
 (('c', 3), ('a', 4)),
 (('c', 3), ('b', 5)),
 (('c', 2), ('e', 1)),
 (('c', 2), ('f', 4)),
 (('a', 4), ('e', 1)),
 (('a', 4), ('f', 4)),
 (('c', 2), ('b', 2)),
 (('a', 4), ('b', 2)),
 (('c', 3), ('e', 1)),
 (('c', 3), ('f', 4)),
 (('c', 3), ('b', 2))]

In [45]:
rdd = sc.parallelize([ ("09-06-2021",123) ,("09-07-2021",133) ,("09-08-2021",142)])
rdd = rdd.zipWithIndex()
rdd = rdd.map(lambda x : (x[1],x[0]))
rdd2 = rdd
rdd.collect()
rdd2 = rdd2.map(lambda b : (b[0]+1,b[1]))
rdd3 = rdd2.join(rdd)
rdd3.collect()

[(1, (('09-06-2021', 123), ('09-07-2021', 133))),
 (2, (('09-07-2021', 133), ('09-08-2021', 142)))]

In [46]:
#https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

In [ ]:
#Dolar artışlarını yüzdesel olarak hesaplamak için
#1 -> 2 --> 2-1 /1
#100 -> 120 --> 120-100/100